# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [5]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [6]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [7]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [8]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [10]:
key_ = list(bigram_phrases.vocab.keys())[14598]
print(key_)

a_>


Le dictionnaire indique le score de cette coocurrence :

In [11]:
bigram_phrases.vocab[key_]

1676

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [12]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [13]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [14]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [15]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [16]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle n2 Word2Vec sur ce corpus

In [17]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=3, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=10, # Permet de paralléliser l'entraînement du modèle en 10 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 10min 10s
Wall time: 3min 49s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [18]:
outfile = f"../../data/newspapers.model_n2"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [19]:
model = Word2Vec.load("../../data/newspapers.model_n2")

### Imprimer le vecteur d'un terme

In [20]:
model.wv["metro"]

array([-0.568246  , -0.54830736,  1.4304588 , -0.49253854, -0.04100419,
       -2.6051035 ,  0.31196564,  1.7866768 , -1.2144837 ,  2.1553905 ,
        0.39924413,  0.7838284 , -0.70041335,  1.5271547 , -0.49241516,
       -0.46779105,  0.96171373,  1.3975527 ,  0.5489413 ,  0.3762164 ,
       -0.08368935, -0.94970465,  0.32727697,  1.983942  , -0.6227003 ,
       -1.3051927 , -3.686019  , -1.2819121 ,  1.8466523 ,  2.2346473 ,
       -0.9255948 , -0.14618404], dtype=float32)

### Vérifier la présence des tokens de "transport public" et ses variantes

In [34]:
"transports_publics" in model.wv

True

In [33]:
"transport_public" in model.wv

False

In [38]:
[w for w in model.wv.index_to_key if "transports" in w]

['transports',
 '+_transports',
 'transports_internationaux',
 '[_transports',
 'transports_aeriens',
 'ftransports',
 'vos_transports',
 'transports_reunis',
 'transports_routiers',
 'transports_fluviaux',
 'transports_automobiles',
 'transports_pat',
 'confier_vos_transports',
 'transports_maritimes',
 'transports_camions',
 'et_transports_autotransp',
 'transports_internes',
 'transports_express',
 'transports_coloniaux',
 'transports_economiques',
 'des_transports_intercommunaux',
 'transports_urbains',
 'transports_publics',
 'transports_ferroviaires',
 'transports_terrest',
 'demenagements_transports',
 'transports_internation',
 'transports_terrestres',
 'transports_ennemis',
 'transports_congolais',
 'transports_gratuits',
 'transports_lourds',
 'funerailles_transports',
 'vermeulen_transports_demenagements',
 'transports_demenagements',
 'transports_internat',
 'transports_intern']

In [39]:
[w for w in model.wv.index_to_key if "transport" in w]

['transport',
 'transports',
 'ete_transporte',
 'transporte',
 'transporter',
 'ete_transportee',
 'transporta',
 'fut_transporte',
 'transportee',
 'transportait',
 'transportes',
 'transportant',
 'ont_ete_transportes',
 'shell_transport',
 'transporteur',
 'etre_transporte',
 'fut_transportee',
 'transport_gratuit',
 'transporteurs',
 'transporterent',
 '+_transports',
 'transportees',
 'ont_ete_transportees',
 'avait_ete_transporte',
 'transports_internationaux',
 'fit_transporter',
 'transportaient',
 '[_transports',
 'dut_etre_transporte',
 '+_transport',
 'transports_aeriens',
 'fait_transporter',
 'transportent',
 'etre_transportee',
 'furent_transportes',
 'etre_transportes',
 'avait_ete_transportee',
 'transportera',
 'ils_ont_ete_transportes',
 'ftransports',
 'vos_transports',
 'transports_reunis',
 'transport_aerien',
 'victime_fut_transportee',
 'transports_routiers',
 'transports_fluviaux',
 'transportable',
 'transports_automobiles',
 'transport_assure',
 'immediatemen

### Calculer la similarité entre deux termes

In [21]:
model.wv.similarity("train", "tram")

0.5912483

In [22]:
model.wv.similarity("metro", "train")

0.4600343

In [23]:
model.wv.similarity("fiat", "opel")

0.84171283

In [24]:
model.wv.similarity("transport", "public")

0.40372035

In [36]:
model.wv.similarity("transports", "publics")

0.6101042

In [25]:
model.wv.similarity("voiture", "fiat")

0.27896893

In [26]:
model.wv.similarity("voiture", "opel")

0.1696556

### Chercher les mots les plus proches d'un terme donné

In [27]:
model.wv.most_similar("metro", topn=10)

[('mercure', 0.8099485635757446),
 ('texas', 0.8070723414421082),
 ('guano', 0.7944544553756714),
 ('rallye', 0.7916650176048279),
 ('marathon', 0.7666881680488586),
 ('coucou', 0.7595226764678955),
 ('lotus', 0.7531875371932983),
 ('lyonnais', 0.7461369037628174),
 ('punch', 0.7435508370399475),
 ('couchant', 0.7434605360031128)]

In [48]:
model.wv.most_similar(
    negative=['metro']
)

[('deslro', 0.5972616076469421),
 ('ne_justifiait', 0.5758865475654602),
 ('snns', 0.5434922575950623),
 ('deaire', 0.538543164730072),
 ('menca', 0.534765362739563),
 ('derire', 0.5186828970909119),
 ('mari_travail', 0.5174874663352966),
 ('fslre', 0.5158072113990784),
 ('aurais_pu', 0.5139988660812378),
 ('voul', 0.5088689923286438)]

In [49]:
model.wv.most_similar(
    negative=['aeriens', 'routiers', 'fluviaux']
)

[('pum', 0.4646149277687073),
 ('eugenie', 0.4440712332725525),
 ('mademoiselle', 0.4401196241378784),
 ('lobar', 0.4377756416797638),
 ('cherie', 0.43254414200782776),
 ('jolie', 0.4277503490447998),
 ('desnoyers', 0.42642858624458313),
 ('rue_duques', 0.4249996840953827),
 ('ma_pauvre', 0.42253419756889343),
 ('rosa', 0.41690629720687866)]

In [28]:
model.wv.most_similar("fiat", topn=10)

[('peugeot', 0.8923713564872742),
 ('chevrolet', 0.8881485462188721),
 ('lancia', 0.8660717606544495),
 ('cabriolet', 0.8603569865226746),
 ('ford', 0.8576937913894653),
 ('citroen', 0.8557456135749817),
 ('fiat_500', 0.8504438996315002),
 ('chrysler', 0.8473367094993591),
 ('bulck', 0.8471443057060242),
 ('dkw', 0.8466489315032959)]

In [50]:
model.wv.most_similar(
    negative=['fiat']
)

[('adoptee_par', 0.5981795191764832),
 ('estimant_que', 0.5901011824607849),
 ('inspiree_par', 0.5859140753746033),
 ('formulees_par', 0.5803821086883545),
 ('dictee_par', 0.5779798030853271),
 ('reclamee_par', 0.5762003064155579),
 ('basee_sur', 0.5701943635940552),
 ('du_defunt', 0.5689523220062256),
 ('ipar', 0.56477952003479),
 ('formelle', 0.5640994906425476)]

In [29]:
model.wv.most_similar("opel", topn=10)

[('chevrolet', 0.9522774815559387),
 ('taunus', 0.9512304067611694),
 ('chrysler', 0.9495626091957092),
 ('renault', 0.9472226500511169),
 ('panhard', 0.9472215175628662),
 ('dodge', 0.9468609094619751),
 ('morris', 0.9462552666664124),
 ('vauxhall', 0.9431740641593933),
 ('ford', 0.9429470896720886),
 ('studebaker', 0.9427738189697266)]

In [51]:
model.wv.most_similar(
    negative=['opel']
)

[('estimant_que', 0.6603404879570007),
 ('lutte_contre', 0.6259908080101013),
 ('sante_publique', 0.5886390805244446),
 ('discipline', 0.5879874229431152),
 ('ipar', 0.581672728061676),
 ('facon_dont', 0.5768484473228455),
 ('magistrature', 0.5757755637168884),
 ('justice', 0.5719097852706909),
 ('pretendue', 0.5662567615509033),
 ('commission_des_finances', 0.5635051131248474)]

In [30]:
model.wv.most_similar("voiture", topn=10)

[('voilure', 0.9554378986358643),
 ('motocyclette', 0.8800674676895142),
 ('camionnette', 0.8742683529853821),
 ('bicyclette', 0.8700584173202515),
 ('valise', 0.8506015539169312),
 ('cabine', 0.846881628036499),
 ('remorque', 0.8436433672904968),
 ('grue', 0.832355797290802),
 ('jeep', 0.8295406699180603),
 ('malle', 0.8228439688682556)]

In [52]:
model.wv.most_similar(
    negative=['voiture']
)

[('_du', 0.5412017107009888),
 ('6_mhz', 0.520358145236969),
 ('nos_mots_croises', 0.519142746925354),
 ('nationalistes_irlandais', 0.5130373239517212),
 ('sont_pas_rendus', 0.5119305849075317),
 ('rlu', 0.5015339851379395),
 ('fleet_street', 0.4982600212097168),
 ('%,_230', 0.49627649784088135),
 ('dtf', 0.4945685565471649),
 ('coupons_detaches', 0.48882976174354553)]

In [31]:
model.wv.most_similar("tram", topn=10)

[('terminus', 0.8598461151123047),
 ('vicinal', 0.8292694091796875),
 ('tram_vicinal', 0.8146908283233643),
 ('tram_electrique', 0.7936480045318604),
 ('nord', 0.7873116731643677),
 ('canal', 0.7766499519348145),
 ('paro', 0.7565405368804932),
 ('rivage', 0.7452163100242615),
 ('macadam', 0.7412645220756531),
 ('vieux_moulin', 0.736478865146637)]

In [53]:
model.wv.most_similar(
    negative=['tram']
)

[('agreer_mes', 0.587174654006958),
 ('apprecieront', 0.5714657306671143),
 ('qui_souffrez', 0.5595791935920715),
 ('doiit', 0.5577088594436646),
 ('enseignera', 0.5491213202476501),
 ('ne_justifiait', 0.5394203662872314),
 ('avec_infiniment', 0.5358338952064514),
 ('sinceres', 0.5350090861320496),
 ('positifs', 0.5343674421310425),
 ('apprenez', 0.5320518016815186)]

In [32]:
model.wv.most_similar("train", topn=10)

[('bateau', 0.9046051502227783),
 ('vehicule', 0.8728815317153931),
 ('canot', 0.8693628311157227),
 ('renfort', 0.8447500467300415),
 ('parcours', 0.837190568447113),
 ('tramway', 0.8320090174674988),
 ('trajet', 0.8281790614128113),
 ('convoi', 0.8224095106124878),
 ('barrage', 0.8220364451408386),
 ('signal', 0.8174459338188171)]

In [54]:
model.wv.most_similar(
    negative=['train']
)

[('danse_moderne', 0.6131625175476074),
 ('_du', 0.6071618795394897),
 ('adaptation_sociale', 0.5980402827262878),
 ('desire_187', 0.5699783563613892),
 ('accouch', 0.5630399584770203),
 ('10166', 0.5591892004013062),
 ("(('", 0.553024172782898),
 ('9933', 0.5516930222511292),
 ('./.-', 0.5506836175918579),
 ('vcc', 0.5481482744216919)]

### Faire des recherches complexes à travers l'espace vectoriel

In [ ]:
print(model.wv.most_similar(
    positive=['transports_publics', 'train'],
    negative=['aerien', 'routier', 'fluvial']
))

[('jra', 0.5223293900489807), ('13ii', 0.49160000681877136), ('moyenne_id', 0.49053964018821716), ('guerison', 0.48004117608070374), ('persister', 0.47517189383506775), ('medis', 0.4747520387172699), ('jrs', 0.47189024090766907), (',-:', 0.4660273790359497), ('pret_immediat', 0.4624255299568176), ('modique_somme', 0.45959919691085815)]


In [56]:
model.wv.most_similar(positive=["metro", "bus"], negative=["voiture", "taxi"])

[('bolgo', 0.7236853837966919),
 ('concordia', 0.7093651294708252),
 ('chili', 0.6913998126983643),
 ('bank', 0.6899431347846985),
 ('molinos', 0.6818704605102539),
 ('credit_mobilier', 0.6808441281318665),
 ('phenix', 0.674060583114624),
 ('pacific', 0.6689425110816956),
 ('beceka', 0.6672743558883667),
 ('205', 0.6618202328681946)]

In [58]:
model.wv.most_similar(
    positive=["metro", "bus", "tram", "train"],
    negative=["voiture", "taxi", "vehicule", "opel", "fiat"]
)


[('bolgo', 0.6448934078216553),
 ('vietnam', 0.6404877305030823),
 ('dominicain', 0.6299099326133728),
 ('laos', 0.6147130131721497),
 ('rwanda', 0.6116872429847717),
 ('burundi', 0.6065951585769653),
 ('transvaal', 0.602756679058075),
 ('fienu', 0.6011660099029541),
 ('paraguay', 0.5995774865150452),
 ('bastion', 0.5968899130821228)]